In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
censuses_data = pd.read_csv("./CSV/censuses.csv")
# censuses_data = censuses_data[censuses_data["census_year"] == 1900]
censuses_data.count()

line_number               4090
house_number              4090
street_name               4090
last_name                 4090
given_name                4090
relation                  4090
color_race                4090
sex                       4090
age_at_last_birthday      4090
occupation                4090
own_or_rent               4090
owned_free_or_mortgage    4090
house_or_farm             4090
my_notes                  4090
file_name                 4090
census_year               4090
dtype: int64

In [3]:
# Separate out the occupation which are null/ none (LATER we can test our model again these cases)
occupation_missing_mask = (censuses_data["occupation"].str.contains('None')) | (censuses_data["occupation"].str.contains('Null'))
df_missing_occupation = censuses_data[occupation_missing_mask]

df_missing_occupation.count()

line_number               1389
house_number              1389
street_name               1389
last_name                 1389
given_name                1389
relation                  1389
color_race                1389
sex                       1389
age_at_last_birthday      1389
occupation                1389
own_or_rent               1389
owned_free_or_mortgage    1389
house_or_farm             1389
my_notes                  1389
file_name                 1389
census_year               1389
dtype: int64

In [4]:
# prepare new df where occupation column is All labelled
df_all_occupation = censuses_data[occupation_missing_mask == False]

# include following features that MAY influence occupation field -
# color_race, sex, age_at_last_birthday, occupation
newdata = df_all_occupation.loc[:,["color_race","sex","age_at_last_birthday", "occupation"]]
newdata.count()

color_race              2701
sex                     2701
age_at_last_birthday    2701
occupation              2701
dtype: int64

In [5]:
newdata.groupby(['occupation']).count().sort_values(by=['occupation'])

,color_race,sex,age_at_last_birthday
occupation,,,
Laundress At Home,1,1,1
Waitress At Restaurant,1,1,1
Abstracter At Loan And Investments,1,1,1
Accountant At Light And Power Company,1,1,1
Actor At Traveling Show,2,2,2
Actress At Theatrical Company,1,1,1
Agent (Machines),1,1,1
Agent At Real Estate,2,2,2
Agent At Real Estate,1,1,1


In [6]:
from fuzzywuzzy import fuzz, process

D:\Users\kshah\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
correct_occupations=["Agent","Barber","Bakery","Beauty","Cook","Clerk","Cleaner","Drive","Express", "Fireman","Foreman","Helper",\
                     "Labor","Laundr","Machinist","Manager","Maid",\
                     "Paint","Plumber","Porter","Railway","Teamster","Wait","Washwoman","Worker", "Wringer","Yard" ]
# replace Yard by Yardman
# replace wait by waiter/ Waitress
# replace Laundr by Laundress

In [8]:
def fuzzy_match_occ(occupation):
    global df_occu
    new_name, score = process.extractOne(occupation, correct_occupations)
    if(score > 70):
        # Good match case
        if(new_name in df_occu):
        # entry found in corrected occupation then append it
            if not(occupation in df_occu[new_name]):
                df_occu[new_name].append(occupation)
        else:
            # make entry for the new_name found
            df_occu[new_name] = [occupation,]
    else:
        # Poor match score 
        if not(occupation in df_occu):
            # make entry for the new_name found
            df_occu[occupation] = [occupation,]
    
    return df_occu

In [9]:
df_occu = dict()
data = zip(*newdata['occupation'].apply(fuzzy_match_occ))

In [10]:
print(len(df_occu))
print("*****************************************")
for row in df_occu:
    print(row, df_occu[row])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print("Matching similar occupation count is ", len(df_occu[row]))
    print("---------------------------------")

490
*****************************************
Servant ['Servant']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Drayman ['Drayman']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Laundr ['Laundress', 'Laundryman', 'Landlord', 'Ironer At Laundry', 'Washwoman At Laundry', 'Wringer Man At Laundry', 'Laundress At Home', 'Laundress At Private Families', 'Starcher At Laundry', 'Sollicitor At Laundry', 'Laundress At Private Family', 'Show Boss At Laundry', 'Washer At Laundry', 'Laundress At Hotel', 'Dresser At Laundry', 'Sorter At Laundry', 'Market At Laundry', 'Stableman At Laundry', 'Worker At Laundry', 'Folder At Laundry', 'Porter At Laundry', 'Presser At Laundry', 'Finisher At Laundry', 'Wringer At Laundry', 'Marker At Laundry', 'Presser At Lauyndry', 'Shirt Finisher At Laundry', 'Laundress At Laundry', 'Finisher At Lauyndry', 'Stamper At Laundry', 'Seamstress At Laun

Accountant At Light And Power Company ['Accountant At Light And Power Company']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Saleslady At Seed House ['Saleslady At Seed House']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Stenographer At Lawyer'S Office ["Stenographer At Lawyer'S Office"]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Bookkeeper At Department Store ['Bookkeeper At Department Store']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Cashier At Department Store ['Cashier At Department Store']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Soda Dispenser At Drug Store ['Soda Dispenser At Drug Store']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  

Manicurist At Homes ['Manicurist At Homes']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Chauffeur  At Plate Glass Co. ['Chauffeur  At Plate Glass Co.']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Retired Mechanic At House ['Retired Mechanic At House']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Musician At Violin In Band ['Musician At Violin In Band']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Seamstress At Private Families ['Seamstress At Private Families']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Owner At Transfer Wagon ['Owner At Transfer Wagon']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Roofing At H